<a href="https://colab.research.google.com/github/DaemonOnCode/Breast-cancer-detection-using-PyTorch/blob/main/Breast_cancer_detection_using_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
#0) Prepare data
bc = datasets.load_breast_cancer()
x, y = bc.data, bc.target

n_samples, n_features = x.shape
print(n_samples, n_features)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)

#scale
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

#1) model
#f = wx + b, sigmoid at the end
class LogisticRegression(nn.Module):
  def __init__(self, n_input_features):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(n_input_features, 1)

  def forward(self, x):
    y_predicted = torch.sigmoid(self.linear(x))
    return y_predicted

model = LogisticRegression(n_features)

#2) loss and optimizer

lr = 10
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

#3) training loop
num_epochs = 1000
for epoch in range(num_epochs):
  y_predicted = model(x_train)
  loss = criterion(y_predicted, y_train)
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()

  if (epoch+1) % 100 == 0:
    print(f'epoch: {epoch+1}, loss: {loss.item():.4f}')

with torch.no_grad():
  y_predicted = model(x_test)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(y_test).sum() / float (y_test.shape[0])
  print(f'accuracy: {acc*100:.2f}%')

569 30
epoch: 100, loss: 0.0243
epoch: 200, loss: 0.0168
epoch: 300, loss: 0.0141
epoch: 400, loss: 0.0124
epoch: 500, loss: 0.0112
epoch: 600, loss: 0.0102
epoch: 700, loss: 0.0094
epoch: 800, loss: 0.0088
epoch: 900, loss: 0.0082
epoch: 1000, loss: 0.0077
accuracy: 97.37%
